In [1]:
from dotenv import load_dotenv
import pandas as pd
import os
from pymongo import MongoClient

load_dotenv()
cluster_uri = os.environ.get("MONGODB_URI")
client = MongoClient(cluster_uri)
db = client["MSCI446_DB"]

### This module is for loading the data and transforming them into dataframes

In [2]:
# This module converts the objects pulled from Mono DB into a dataframe
collection_forecast = db["Load_Forecast"]
collection_solar = db["Solar_Forecast"]
collection_wind = db["Wind_Forecast"]
collection_gas_price = db["Gas_Prices"]
collection_gen_outages = db["Gen_Outages"]
collection_historical_da_price = db["Historical_DA_Prices"]

documents_forecast = list(collection_forecast.find())
documents_solar = list(collection_solar.find())
documents_wind = list(collection_wind.find())
documents_gas_price = list(collection_gas_price.find())
documents_gen_outages = list(collection_gen_outages.find())
documents_historical_da_price = list(collection_historical_da_price.find())

# Converting documents into dataframes
df_forecast = pd.DataFrame(documents_forecast)
df_solar = pd.DataFrame(documents_solar)
df_wind = pd.DataFrame(documents_wind)
df_gas_price = pd.DataFrame(documents_gas_price)
df_gen_outages = pd.DataFrame(documents_gen_outages)
df_historical_da_price = pd.DataFrame(documents_historical_da_price)

# Dropping the id column
df_forecast.drop('_id', axis=1, inplace=True)
df_solar.drop('_id', axis=1, inplace=True)
df_wind.drop('_id', axis=1, inplace=True)
df_gas_price.drop('_id', axis=1, inplace=True)
df_gen_outages.drop('_id', axis=1, inplace=True)
df_historical_da_price.drop('_id', axis=1, inplace=True)

print(df_forecast.head())
print(df_solar.head())
print(df_wind.head())
print(df_gas_price.head())
print(df_gen_outages.head())
print(df_historical_da_price.head())

  forecast_hour_beginning_ept forecast_area  forecast_load_mw
0        1/1/2024 12:00:00 AM           AEP             14145
1         1/1/2024 1:00:00 AM           AEP             13908
2         1/1/2024 2:00:00 AM           AEP             13765
3         1/1/2024 3:00:00 AM           AEP             13788
4         1/1/2024 4:00:00 AM           AEP             13862
  datetime_beginning_ept    area  solar_generation_mw
0  2/28/2024 11:00:00 PM  MIDATL                  0.0
1  2/28/2024 11:00:00 PM   OTHER                  0.0
2  2/28/2024 11:00:00 PM     RFC                  0.0
3  2/28/2024 11:00:00 PM     RTO                  0.0
4  2/28/2024 11:00:00 PM   SOUTH                  0.0
   datetime_beginning_ept    area  wind_generation_mw
0  12/31/2023 11:00:00 PM  MIDATL              75.609
1  12/31/2023 11:00:00 PM   OTHER              26.714
2  12/31/2023 11:00:00 PM   SOUTH             109.142
3  12/31/2023 11:00:00 PM    WEST            4174.260
4  12/31/2023 11:00:00 PM     RTO 

### This module is for getting unique areas from each dataframe

In [3]:
# Get unique area values from each dataframe
unique_areas_solar = df_solar['area'].unique()
unique_areas_wind = df_wind['area'].unique()
unique_areas_load_forecast = df_forecast['forecast_area'].unique()
unique_areas_historical_da_price = df_historical_da_price['zone'].unique()

# Print the unique area values
print("Unique areas in Solar Forecast table:", unique_areas_solar)
print("Unique areas in Wind Forecast table:", unique_areas_wind)
print("Unique areas in Load Forecast:", unique_areas_load_forecast)
print("Unique areas in Historical DA Price:", unique_areas_historical_da_price)

Unique areas in Solar Forecast table: ['MIDATL' 'OTHER' 'RFC' 'RTO' 'SOUTH' 'WEST']
Unique areas in Wind Forecast table: ['MIDATL' 'OTHER' 'SOUTH' 'WEST' 'RTO' 'RFC']
Unique areas in Load Forecast: ['AEP' 'APS' 'ATSI' 'COMED' 'DAY' 'DEOK' 'DOM' 'DUQ' 'EKPC' 'MIDATL' 'RTO']
Unique areas in Historical DA Price: ['DPL']


### This module is for renaming column headings to common datetime heading

In [4]:
# Renaming date-time columns in various dataframes to a common date-time column heading
df_forecast.rename(columns={'forecast_hour_beginning_ept': 'datetime_beginning_ept'}, inplace=True)
df_gen_outages.rename(columns={'forecast_date': 'datetime_beginning_ept'}, inplace=True)
df_gas_price.rename(columns={'Date': 'datetime_beginning_ept'}, inplace=True)

### This module is for pivoting dataframes and changing column headings for merge

In [5]:
# Instead of dropping the duplicate rows, I just averaged them and since they're the same their value shouldn't change
wind_pivot = df_wind.pivot_table(index='datetime_beginning_ept', columns='area', values='wind_generation_mw', aggfunc='mean')
solar_pivot = df_solar.pivot_table(index='datetime_beginning_ept', columns='area', values='solar_generation_mw', aggfunc='mean')
load_forecast_pivot = df_forecast.pivot_table(index='datetime_beginning_ept', columns ='forecast_area', values = 'forecast_load_mw', aggfunc = 'mean')
historical_da_pivot = df_historical_da_price.pivot_table(index='datetime_beginning_ept', columns ='zone', values = 'total_lmp_da', aggfunc = 'mean')

# Renaming column headings to differentiate from other columns
load_forecast_pivot = load_forecast_pivot.rename(columns={col: f"{col}_forecast" for col in load_forecast_pivot.columns if col != 'datetime_beginning_ept'})
historical_da_pivot = historical_da_pivot.rename(columns={col: f"{col}_historical_da" for col in historical_da_pivot.columns if col != 'datetime_beginning_ept'})

### This module is for preparing the gen_outages dataset

In [6]:
# Converting the column into date time
df_gen_outages['datetime_beginning_ept'] = pd.to_datetime(df_gen_outages['datetime_beginning_ept'])

# The gen_outage dataframe has duplicate dates (rows) that and their values need to be averaged 
df_gen_outages = df_gen_outages.groupby('datetime_beginning_ept').mean().reset_index()

# Filtering out rows where the date is earlier than 2022
df_gen_outages = df_gen_outages[df_gen_outages['datetime_beginning_ept'].dt.year >= 2022]
# Set the 'datetime_beginning_ept' column as the index, resample to hourly increments, and forward fill the gas prices
df_gen_outages_index = df_gen_outages.set_index('datetime_beginning_ept').resample('H').ffill()

# Reset the index to make 'datetime_beginning_ept' a column again
df_gen_outages_index.reset_index(inplace = True)

# Convert the 'datetime_beginning_ept' column from datetime to string
df_gen_outages_index['datetime_beginning_ept'] = df_gen_outages_index['datetime_beginning_ept'].dt.strftime('%d/%m/%Y %H:%M:%S %p')

print(df_gen_outages_index.tail())

       datetime_beginning_ept  forecast_gen_outage_mw_rto  \
21116  30/05/2024 20:00:00 PM                     12119.0   
21117  30/05/2024 21:00:00 PM                     12119.0   
21118  30/05/2024 22:00:00 PM                     12119.0   
21119  30/05/2024 23:00:00 PM                     12119.0   
21120  31/05/2024 00:00:00 AM                     11724.0   

       forecast_gen_outage_mw_west  forecast_gen_outage_mw_other  
21116                       5494.0                        6625.0  
21117                       5494.0                        6625.0  
21118                       5494.0                        6625.0  
21119                       5494.0                        6625.0  
21120                       5314.0                        6410.0  


/var/folders/8z/yxhm1c6x4bvg_s6d6w0cl6_80000gn/T/ipykernel_37657/3941139758.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_gen_outages['datetime_beginning_ept'] = pd.to_datetime(df_gen_outages['datetime_beginning_ept'])
/var/folders/8z/yxhm1c6x4bvg_s6d6w0cl6_80000gn/T/ipykernel_37657/3941139758.py:10: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_gen_outages_index = df_gen_outages.set_index('datetime_beginning_ept').resample('H').ffill()


### This module is for preparing the gas_price dataset

In [7]:
# Changing the 'datetime_beginning_ept' column into datetime format
df_gas_price['datetime_beginning_ept'] = pd.to_datetime(df_gas_price['datetime_beginning_ept'])

# Filtering out rows where the date is earlier than 2022
df_gas_price = df_gas_price[df_gas_price['datetime_beginning_ept'].dt.year >= 2022]

# Set the 'datetime_beginning_ept' column as the index, resample to hourly increments, and forward fill the gas prices
df_gas_price_index = df_gas_price.set_index('datetime_beginning_ept').resample('H').ffill()

# Reset the index to make 'datetime_beginning_ept' a column again
df_gas_price_index.reset_index(inplace=True)

# Convert the 'datetime_beginning_ept' column from datetime to string
df_gas_price_index['datetime_beginning_ept'] = df_gas_price_index['datetime_beginning_ept'].dt.strftime('%d/%m/%Y %H:%M:%S %p')

# Print the first few rows of the resampled DataFrame to verify the results
print(df_gas_price_index)

       datetime_beginning_ept  \
0      01/01/2022 00:00:00 AM   
1      01/01/2022 01:00:00 AM   
2      01/01/2022 02:00:00 AM   
3      01/01/2022 03:00:00 AM   
4      01/01/2022 04:00:00 AM   
...                       ...   
18884  26/02/2024 20:00:00 PM   
18885  26/02/2024 21:00:00 PM   
18886  26/02/2024 22:00:00 PM   
18887  26/02/2024 23:00:00 PM   
18888  27/02/2024 00:00:00 AM   

       Henry Hub Natural Gas Spot Price (Dollars per Million Btu)  
0                                                   3.82           
1                                                   3.82           
2                                                   3.82           
3                                                   3.82           
4                                                   3.82           
...                                                  ...           
18884                                               1.65           
18885                                               1.65   

/var/folders/8z/yxhm1c6x4bvg_s6d6w0cl6_80000gn/T/ipykernel_37657/1935191083.py:8: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_gas_price_index = df_gas_price.set_index('datetime_beginning_ept').resample('H').ffill()


### This module is for preparing load_forecast dataset

In [8]:
# Converting the column into date time
load_forecast_pivot.index = pd.to_datetime(load_forecast_pivot.index)

# The gen_outage dataframe has duplicate dates (rows) that and their values need to be averaged
load_forecast_pivot = load_forecast_pivot.groupby(load_forecast_pivot.index).mean()

#Filtering out rows where the date is earlier than 2022
load_forecast_pivot = load_forecast_pivot[load_forecast_pivot.index.year >= 2022]

# Set the 'datetime_beginning_ept' column as the index, resample to hourly increments, and forward fill the gas prices
df_forecast_resampled = load_forecast_pivot.resample('H').ffill()

# Reset the index to make 'datetime_beginning_ept' a column again
df_forecast_resampled.reset_index(inplace=True)

# Convert the 'datetime_beginning_ept' column from datetime to string
df_forecast_resampled['datetime_beginning_ept'] = df_forecast_resampled['datetime_beginning_ept'].dt.strftime('%d/%m/%Y %H:%M:%S %p')

print(df_forecast_resampled)

/var/folders/8z/yxhm1c6x4bvg_s6d6w0cl6_80000gn/T/ipykernel_37657/242692983.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_forecast_resampled = load_forecast_pivot.resample('H').ffill()


forecast_area  datetime_beginning_ept  AEP_forecast  APS_forecast  \
0              01/01/2022 00:00:00 AM  11448.600000   4509.200000   
1              01/01/2022 01:00:00 AM  11122.600000   4378.000000   
2              01/01/2022 02:00:00 AM  10934.600000   4292.000000   
3              01/01/2022 03:00:00 AM  10830.200000   4258.000000   
4              01/01/2022 04:00:00 AM  10876.833333   4266.166667   
...                               ...           ...           ...   
18403          06/02/2024 19:00:00 PM  16341.333333   6669.222222   
18404          06/02/2024 20:00:00 PM  16403.888889   6662.222222   
18405          06/02/2024 21:00:00 PM  16210.666667   6538.000000   
18406          06/02/2024 22:00:00 PM  15771.000000   6352.400000   
18407          06/02/2024 23:00:00 PM  15351.700000   6151.600000   

forecast_area  ATSI_forecast  COMED_forecast  DAY_forecast  DEOK_forecast  \
0                5813.000000     9282.000000   1483.000000    2242.400000   
1                

### This module is for preparing the solar table

In [9]:
# Converting the column into date time
solar_pivot.index = pd.to_datetime(solar_pivot.index)

# The gen_outage dataframe has duplicate dates (rows) that and their values need to be averaged
solar_pivot = solar_pivot.groupby(solar_pivot.index).mean()

# The gen_outage dataframe has duplicate dates (rows) that and their values need to be averaged
solar_pivot = solar_pivot.groupby(solar_pivot.index).mean()

#Filtering out rows where the date is earlier than 2022
solar_pivot = solar_pivot[solar_pivot.index.year >= 2022]

# Set the 'datetime_beginning_ept' column as the index, resample to hourly increments, and forward fill the gas prices
df_solar_resampled = solar_pivot.resample('H').ffill()

# Reset the index to make 'datetime_beginning_ept' a column again
df_solar_resampled.reset_index(inplace=True)

# Convert the 'datetime_beginning_ept' column from datetime to string
df_solar_resampled['datetime_beginning_ept'] = df_solar_resampled['datetime_beginning_ept'].dt.strftime('%d/%m/%Y %H:%M:%S %p')

print(df_solar_resampled)

area   datetime_beginning_ept  MIDATL  OTHER    RFC    RTO  SOUTH   WEST
0      01/01/2022 00:00:00 AM  -1.333    0.0 -1.581 -5.175 -3.594 -0.248
1      01/01/2022 01:00:00 AM  -1.323    0.0 -1.564 -5.139 -3.575 -0.241
2      01/01/2022 02:00:00 AM  -1.355    0.0 -1.601 -5.095 -3.494 -0.246
3      01/01/2022 03:00:00 AM  -1.313    0.0 -1.550 -5.115 -3.565 -0.237
4      01/01/2022 04:00:00 AM  -1.363    0.0 -1.600 -5.237 -3.637 -0.237
...                       ...     ...    ...    ...    ...    ...    ...
18931  28/02/2024 19:00:00 PM   0.000    0.0  0.000  0.000  0.000  0.000
18932  28/02/2024 20:00:00 PM   0.000    0.0  0.000  0.000  0.000  0.000
18933  28/02/2024 21:00:00 PM   0.000    0.0  0.000  0.000  0.000  0.000
18934  28/02/2024 22:00:00 PM   0.000    0.0  0.000  0.000  0.000  0.000
18935  28/02/2024 23:00:00 PM   0.000    0.0  0.000  0.000  0.000  0.000

[18936 rows x 7 columns]


/var/folders/8z/yxhm1c6x4bvg_s6d6w0cl6_80000gn/T/ipykernel_37657/1385797169.py:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_solar_resampled = solar_pivot.resample('H').ffill()


### This module is for preparing the wind table

In [10]:
# Converting the column into date time
wind_pivot.index = pd.to_datetime(wind_pivot.index)

# The gen_outage dataframe has duplicate dates (rows) that and their values need to be averaged
wind_pivot = wind_pivot.groupby(wind_pivot.index).mean()

# The gen_outage dataframe has duplicate dates (rows) that and their values need to be averaged
wind_pivot = wind_pivot.groupby(wind_pivot.index).mean()

#Filtering out rows where the date is earlier than 2022
wind_pivot = wind_pivot[wind_pivot.index.year >= 2022]

# Set the 'datetime_beginning_ept' column as the index, resample to hourly increments, and forward fill the gas prices
df_wind_resampled = wind_pivot.resample('H').ffill()

# Reset the index to make 'datetime_beginning_ept' a column again
df_wind_resampled.reset_index(inplace=True)

# Convert the 'datetime_beginning_ept' column from datetime to string
df_wind_resampled['datetime_beginning_ept'] = df_wind_resampled['datetime_beginning_ept'].dt.strftime('%d/%m/%Y %H:%M:%S %p')

print(df_wind_resampled)

/var/folders/8z/yxhm1c6x4bvg_s6d6w0cl6_80000gn/T/ipykernel_37657/100216901.py:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_wind_resampled = wind_pivot.resample('H').ffill()


area   datetime_beginning_ept   MIDATL   OTHER       RFC       RTO    SOUTH  \
0      01/01/2022 00:00:00 AM  444.796     NaN  3543.618  3737.387  193.769   
1      01/01/2022 01:00:00 AM  496.060     NaN  3319.472  3543.083  223.611   
2      01/01/2022 02:00:00 AM  600.131     NaN  3415.238  3624.553  209.315   
3      01/01/2022 03:00:00 AM  616.684     NaN  3459.549  3628.006  168.457   
4      01/01/2022 04:00:00 AM  570.549     NaN  3570.517  3705.773  135.256   
...                       ...      ...     ...       ...       ...      ...   
18979  01/03/2024 19:00:00 PM  740.607  25.902  3205.173  3411.118  205.945   
18980  01/03/2024 20:00:00 PM  817.800  24.567  3404.145  3600.823  196.678   
18981  01/03/2024 21:00:00 PM  807.747  23.893  3484.830  3711.032  226.202   
18982  01/03/2024 22:00:00 PM  795.035  22.740  3254.662  3433.766  179.104   
18983  01/03/2024 23:00:00 PM  760.054  21.582  3058.629  3264.883  206.254   

area       WEST  
0      3098.822  
1      2823.412

### This module is for preparing the historical_da table

In [11]:
# Converting the column into date time
historical_da_pivot.index = pd.to_datetime(historical_da_pivot.index)

# The gen_outage dataframe has duplicate dates (rows) that and their values need to be averaged
historical_da_pivot = historical_da_pivot.groupby(historical_da_pivot.index).mean()

# The gen_outage dataframe has duplicate dates (rows) that and their values need to be averaged
historical_da_pivot = historical_da_pivot.groupby(historical_da_pivot.index).mean()

#Filtering out rows where the date is earlier than 2022
historical_da_pivot = historical_da_pivot[historical_da_pivot.index.year >= 2022]

# Set the 'datetime_beginning_ept' column as the index, resample to hourly increments, and forward fill the gas prices
df_historical_da_resampled = historical_da_pivot.resample('H').ffill()

# Reset the index to make 'datetime_beginning_ept' a column again
df_historical_da_resampled.reset_index(inplace=True)

# Convert the 'datetime_beginning_ept' column from datetime to string
df_historical_da_resampled['datetime_beginning_ept'] = df_historical_da_resampled['datetime_beginning_ept'].dt.strftime('%d/%m/%Y %H:%M:%S %p')

print(df_historical_da_resampled)

/var/folders/8z/yxhm1c6x4bvg_s6d6w0cl6_80000gn/T/ipykernel_37657/790701905.py:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_historical_da_resampled = historical_da_pivot.resample('H').ffill()


zone   datetime_beginning_ept  DPL_historical_da
0      18/02/2022 00:00:00 AM              38.95
1      18/02/2022 01:00:00 AM              36.43
2      18/02/2022 02:00:00 AM              36.36
3      18/02/2022 03:00:00 AM              36.59
4      18/02/2022 04:00:00 AM              38.60
...                       ...                ...
17252  06/02/2024 20:00:00 PM              34.93
17253  06/02/2024 21:00:00 PM              32.97
17254  06/02/2024 22:00:00 PM              30.20
17255  06/02/2024 23:00:00 PM              27.97
17256  07/02/2024 00:00:00 AM              28.31

[17257 rows x 2 columns]


### This module merges the dataframes into one

In [12]:
# Merge the pivoted DataFrames along the dates
merged_df = pd.merge(df_solar_resampled, df_wind_resampled, on='datetime_beginning_ept', suffixes=('_solar', '_wind'))

merged_df = pd.merge(merged_df, df_forecast_resampled, on = 'datetime_beginning_ept')

merged_df = pd.merge(merged_df, df_historical_da_resampled, on = 'datetime_beginning_ept')

merged_df = pd.merge(merged_df, df_gen_outages_index, on = 'datetime_beginning_ept')

merged_df = pd.merge(merged_df, df_gas_price_index, on = 'datetime_beginning_ept')

print(merged_df)


       datetime_beginning_ept  MIDATL_solar  OTHER_solar  RFC_solar  \
0      18/02/2022 00:00:00 AM         0.282          0.0      0.041   
1      18/02/2022 01:00:00 AM         0.246          0.0     -0.006   
2      18/02/2022 02:00:00 AM         0.300          0.0      0.060   
3      18/02/2022 03:00:00 AM         0.254          0.0      0.015   
4      18/02/2022 04:00:00 AM         0.270          0.0      0.034   
...                       ...           ...          ...        ...   
17251  06/02/2024 19:00:00 PM        -2.886          0.0     -5.653   
17252  06/02/2024 20:00:00 PM        -1.944          0.0     -4.737   
17253  06/02/2024 21:00:00 PM        -1.923          0.0     -3.733   
17254  06/02/2024 22:00:00 PM        -1.954          0.0     -3.767   
17255  06/02/2024 23:00:00 PM        -1.936          0.0     -5.747   

       RTO_solar  SOUTH_solar  WEST_solar  MIDATL_wind  OTHER_wind  RFC_wind  \
0         -4.019       -4.061      -0.240      957.466         NaN 